In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

from config import password

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-02 07:24:04--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.51MB/s    in 0.2s    

2022-06-02 07:24:04 (5.51 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|                Y|
|R2HBOEM8LE9928|          5|            0|          0|   N|                Y|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y|
|R1EBPM82ENI67M|          1|            0|          0|   N|                Y|
|R372S58V6D11AT|          5|            1|          1|   N|                Y|
|R1A4514XOYI1PD|          5|            1|          1|   N|                Y|
|R20D9EHB7N20V6|          5|            0|          0|   N|                Y|
|R1WUTD8MVSROJU|          5|            0|          0|   N|                Y|
|R1QCYLT25812DM|          4|            0|          0|   N|                Y|
| R904DQPBCEM7A|          4|            0|          0|   N|     

1. Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20

In [15]:
twenty_plus_votes = vine_df.filter("total_votes>20")
twenty_plus_votes.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|                Y|
|R274AKHXHYGIN6|          5|           87|        100|   N|     

2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%

In [52]:
twenty_plus_votes = twenty_plus_votes.withColumn('helpful_percent', twenty_plus_votes['helpful_votes']/twenty_plus_votes['total_votes']*100)
twenty_plus_votes = twenty_plus_votes.withColumn("helpful_percent",twenty_plus_votes.helpful_percent.cast('float'))
twenty_plus_votes.show(20)
helpful_twenty_plus_votes = twenty_plus_votes.filter(twenty_plus_votes.helpful_percent>50)
helpful_twenty_plus_votes.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+---------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+---------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|      82.608696|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|          100.0|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|      79.245285|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|          100.0|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|          100.0|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|           80.0|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|       91.42857|
|R3MUBV21QV0IJK|          3|           77|         84|   N| 

3. Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

In [53]:
paid_helpful_twenty_plus_votes = twenty_plus_votes.filter(twenty_plus_votes.vine=='Y')
paid_helpful_twenty_plus_votes.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+---------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+---------------+
| R82QWN2X2OCHB|          5|          176|        208|   Y|                N|       84.61539|
|R1UYHBYE6790BU|          5|           44|         53|   Y|                N|       83.01887|
|R2J3YLX1L4EH2B|          5|          299|        321|   Y|                N|      93.146416|
|R3QDI539WTXKE2|          5|           26|         32|   Y|                N|          81.25|
| RQTPWY6ND2NRV|          4|           37|         37|   Y|                N|          100.0|
| RQZSTE0KOBY2G|          4|           98|        109|   Y|                N|       89.90826|
| RF2RNZEJO0K1N|          4|           23|         26|   Y|                N|       88.46154|
| ROB6VOW41E8O5|          4|          155|        172|   Y| 

4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

In [54]:
unpaid_helpful_twenty_plus_votes = twenty_plus_votes.filter(twenty_plus_votes.vine=='N')
unpaid_helpful_twenty_plus_votes.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+---------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+---------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|      82.608696|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|          100.0|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|      79.245285|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|          100.0|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|          100.0|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|           80.0|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|       91.42857|
|R3MUBV21QV0IJK|          3|           77|         84|   N| 

5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [55]:
vine_total_counts = helpful_twenty_plus_votes.groupby("vine").agg({"vine":"count"}).withColumnRenamed("count(vine)", "total_reviews_count")
vine_total_counts.show()

+----+-------------------+
|vine|total_reviews_count|
+----+-------------------+
|   Y|               1029|
|   N|              46593|
+----+-------------------+



In [56]:
fivestar_reviews = helpful_twenty_plus_votes.filter(helpful_twenty_plus_votes.star_rating=='5')
fivestar_reviews.show(20)
vine_fivestar_counts = fivestar_reviews.groupby("vine").agg({"vine":"count"}).withColumnRenamed("count(vine)", "fivestar_reviews_count")
vine_fivestar_counts.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+---------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|      82.608696|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|          100.0|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|           80.0|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|                Y|       97.72727|
|R274AKHXHYGIN6|          5|           87|        100|   N|                N|           87.0|
| RZETLPHZJFYB8|          5|           23|         24|   N|                Y|      95.833336|
|R1Y05HDL2AWCSA|          5|          140|        145|   N|                Y|       96.55173|
| RPTN47ZNQBPWE|          5|           20|         24|   N| 

In [58]:
vine_reviews_summary = vine_total_counts.join(vine_fivestar_counts, ("vine"))
vine_reviews_summary = vine_reviews_summary.withColumnRenamed('vine', 'incentive')
vine_incentive={'Y':'Paid','N':'Unpaid'}
vine_reviews_summary = vine_reviews_summary.rdd.map(lambda x: (vine_incentive[x.incentive], x.fivestar_reviews_count, x.total_reviews_count) ).toDF(["incentive","fivestar_reviews_count","total_reviews_count"])
vine_reviews_summary = vine_reviews_summary.withColumn('fivestar_reviews_percent', vine_reviews_summary['fivestar_reviews_count']/vine_reviews_summary['total_reviews_count']*100)
vine_reviews_summary = vine_reviews_summary.withColumn("fivestar_reviews_percent",vine_reviews_summary.fivestar_reviews_percent.cast('float'))
vine_reviews_summary.show()

+---------+----------------------+-------------------+------------------------+
|incentive|fivestar_reviews_count|total_reviews_count|fivestar_reviews_percent|
+---------+----------------------+-------------------+------------------------+
|     Paid|                   440|               1029|                42.75996|
|   Unpaid|                 21710|              46593|               46.594982|
+---------+----------------------+-------------------+------------------------+

